<a id="CELL1"></a>
## CELL 1 


In [552]:
"""CELL 1
builds and returns a database session
local assumes a psql instance in a local docker container
only postgres database is supported for configuration_application at this time
"""
"""
gets env-based configuration secret
returns a session to the configuration db
for dev env it pre-populates the database with helper and seed data
"""
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

2019-07-29 11:58:00,917 - core.helpers.session_helper.SessionHelper - INFO - Creating session for dev environment...
2019-07-29 11:58:00,986 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating administrator mocks.
2019-07-29 11:58:01,001 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating administrator mocks.
2019-07-29 11:58:01,002 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pharmaceutical company mocks.
2019-07-29 11:58:01,009 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating pharmaceutical company mocks.
2019-07-29 11:58:01,010 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating brand mocks.
2019-07-29 11:58:01,016 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating brand mocks.
2019-07-29 11:58:01,017 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating segment mocks.
2019-07-29 11:58:0

## CONFIGURATION - PLEASE TOUCH
### <font color=pink>This cell will be off in production as configurations will come from the configuration postgres DB</color>

In [533]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
"""
PIPELINE STATE:

raw-->ingest-->master-->enhance-->enrich-->metrics-->dimensional

"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "master" # the state this transform runs in
config_name = "master_patient_status" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. 
# Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_branch = "sun-extract-validation"
# None
# if None, input_branch is automagically set to your working branch
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"

#This contract defines the base of the output structure of data into S3.
#
#contract structure in s3: 
#s3:// {ENV} / {BRANCH} / {PARENT} / {CHILD} / {STATE} / {name of input}
#
#ENV - environment Must be one of development, uat, production.
#Prefixed with integrichain- due to global unique reqirement
#BRANCH - the software branch for development this will be the working pull request (eg pr-225)
#in uat this will be edge, in production this will be master
#PARENT - The top level source identifier
#this is generally the customer (and it is aliased as such) but can be IntegriChain for internal sources,
#or another aggregator for future-proofing
#CHILD - The sub level source identifier, generally the brand (and is aliased as such)
#STATE - One of: raw, ingest, master, enhance, enrich, metrics


### <font color=orange>SETUP - DON'T TOUCH </font>
Populating config mocker based on config parameters...

In [534]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
"""
RETURNS: A list of 2 items: [transformation_id, run_id] where transformation_id corresponds
to the configuration created/found for {transformation} and run_id is a randomly generated 6 digit
number (to avoid publishing to the same place with the same dataset)
"""
transform_id = ids[0]
run_id = ids[1]

2019-07-26 19:40:04,833 - core.logging - DEBUG - Adding/getting mocks for specified configurations...
2019-07-26 19:40:04,863 - core.logging - DEBUG - Done. Creating mock run event and committing results to configuration mocker.


In [547]:
# debug only
#print(transform_id)
#print(run_id)
# 6
# 136126

### <font color=orange>SETUP - DON'T TOUCH </font>
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 


In [537]:
"""************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET, BATCH_JOB_QUEUE
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


In [538]:
#debug only
print(BRANCH_NAME)
print(ENV_BUCKET)
print(BATCH_JOB_QUEUE)
#DC-578_PatientStatus
#ichain-dev
#dev-core

DC-578_PatientStatus
ichain-dev
dev-core


***
# CORE Cartridge Notebook::[master_patient_status]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

## CONFIGURATION - VARIABLES - PLEASE TOUCH

# TRANSFORM

In [539]:
""" 
CONFIGURATION ********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>
e.g.
class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""
"""
imports
"""
import pandas as pd
# getting a logger through the core project helper function
from core.logging import get_logger
logger = get_logger(f"core.transforms.{transform.state}.{transform.name}")

 
class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    ''' 

    ## cols of interest
    ## Field	     Description	Data Type / Format	Comments	Required	            OLD Name in Data Model	New Name in Data Model	<-- Map to	BIPI	Sun	Alkermes - Status
    ## Status Code	Status Code	    X(20)		                    Required if Available	status	                status				
    col_status: str           
    
    
    def master_patient_status(self,df):
        try:        
            go = True # assume things will work
           
            dffails = pd.DataFrame() # initialize df for fails
            
            # master data golden status
            #working draft] Gold Domain of patient status
            logger.info('try:')
            status_dict = {}
            status_dict[1]='ACTIVE'
            status_dict[2]='CANCELLED'
            status_dict[3]='DISCONTINUED'
            status_dict[4]='DENIED'
            status_dict[5]='PENDING'
            
            # store the golden values in a list
            status_list = list(status_dict.values())           
            
            # log metadata 
            
            logger.info('Gold Domain List:{}'.format(status_list))  
            
            # df in
            logger.info('df in {}'.format(df.head()))
            dfSize = df.size
            dfShape = df.shape
            logger.info('input df  size: {} shape: {} {}'.format(dfSize,dfShape[0],dfShape[1])) 
            
            # am I expecting certain column names? YES 
            statusColNameExpected = 'status_code'
            
            logger.info('expecting column name patient status as:{}'.format(statusColNameExpected))
            columnNamesArr = df.columns.values.tolist()
            logger.info('df column names:{}'.format(columnNamesArr))
            
            if statusColNameExpected in columnNamesArr:
                # can we assume anything about raw to ingest data ? 
                # apply Upper Case to col values of interest
                df[statusColNameExpected]= df[statusColNameExpected].apply(lambda x: x.upper() if x is not None else x)   
                # apply strip  to col values of interest
                df[statusColNameExpected]= df[statusColNameExpected].apply(lambda x: x.strip() if x is not None else x)
                # what fails
                dffail = df[~df[statusColNameExpected].isin(status_list)]
                # apply master selection for the column of interest
                # what passes
                df = df[df[statusColNameExpected].isin(status_list)]
                
                # meta data log for what comes out of the function pass df
                dfOutSize = df.size
                dfOutShape = df.shape
                logger.info('out  df  size: {} shape: {} {}'.format(dfOutSize,dfOutShape[0],dfOutShape[1])) 
                
                # meta data log for what comes out of the function fail df
                logger.info('df fail {}'.format(dffail.head()))
                
                dffailSize = dffail.size
                dffailShape = dffail.shape
                logger.info('fail df  size: {} shape: {} {}'.format(dffailSize,dffailShape[0],dfFailShape[1]))                 
            else:
                go = False # something did not work
                logger.exception('expecting column name for patient status if/else exception raise')
                raise Exception("master_patient_status try if else saletermColNameExpected in columnNamesArr")              
        except Exception as e:
            go = False # something did not work
            logger.exception("exception:".format(e))
            pass
            #raise Exception("master_patient_status"+str(e))
        else:
            pass
        finally:
            pass
        return df.copy(),dffails.copy(),go
                

transform = Transform()

### *Please place your value assignments for development below*
### <font color=pink>This cell will be turned off in production, Engineering will set to pull from the configuration</color>

In [540]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

transform.col_status = 'status_code'


### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

## Planned
1. Collect all unique raw patient status instances
2. Auto-map as many raw patient status instances to a defined cleansed data model.
*working draft Gold Domain of patient status:*  

   1. ACTIVE
   
   2. CANCELLED
   
   3. DENIED
   
   4. DISCONTINUED
   
   5. PENDING

3. Process for identifying and manually mapping where auto-map fails.
4. Do not publish un-mapped instances. Drop them, give us the ability to triage and map to gold in a later event.


### FETCH DATA - TOUCH, BUT CAREFULLY
### <font color=pink>This cell will be turned off in production, as the input_contract will be handled by the pipeline</color>

In [541]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""
logger.info("FETCH DATA CELL - TOUCH - This cell will be turned off in production, as the input_contract will be handled by the pipeline. ")

# for testing / development only
run_id = 3

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch,
                                 state=input_state, 
                                 parent=input_pharma, 
                                 child=input_brand, 
                                 dataset=input_name)
run_filter = []
run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[run_id]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!

# bypass/comment out when unit testing individual parquet files
df = input_contract.fetch(filters=run_filter)



2019-07-26 19:41:29,504 - core.transforms.master.master_patient_status - INFO - FETCH DATA CELL - TOUCH - This cell will be turned off in production, as the input_contract will be handled by the pipeline. 
2019-07-26 19:41:29,507 - core.dataset_contract.DatasetContract - INFO - Fetching dataframe from s3 location s3://ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping.


## *<font color=grey>unit test development only*</font>

In [553]:
# temporary cell only for unit test/development
import pyarrow.parquet as pq
import s3fs

def pandas_from_parquet_s3(file_path):  
    s3 = s3fs.S3FileSystem()
    df = (
        pq
        .ParquetDataset(file_path, filesystem=s3)
        .read_pandas()
        .to_pandas()
    )    
    return df

In [439]:
# unit test/development 
# isolate on individual parquet files
#TEST 1
#df = pandas_from_parquet_s3('ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=3/90ca3aa7b0bb4246a281591b013ff54e.parquet')
# TEST 2
#df = pandas_from_parquet_s3('ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=3/d7ad974cef284e19aa7b5ac410220b96.parquet')
# TEST 3
#df = pandas_from_parquet_s3('ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=3/1a6ffd3598d442e38fbba66ea85a55a2.parquet')
# TEST 4
#df = pandas_from_parquet_s3('ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=3/5c00059d9fc04b0e8bc4ce764c50f3fb.parquet')
# TEST 5
# df = pandas_from_parquet_s3('ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=3/6eceb7ce59bd4dec8720316b4209b0e3.parquet')
# THEN ALL TEST use 
# then use the FETCH DATA - TOUCH, BUT CAREFULLY CELL

In [550]:
# unit test/development
# before shot unit testing only
dfSize = df.size
dfShape = df.shape
print('size: {} shape: {} {}'.format(dfSize,dfShape[0],dfShape[1])) 

size: 1760904 shape: 24457 72


In [548]:
# unit test/development
# needed to see the col(s) of interest
# pd.set_option('display.max_columns', 50)

In [551]:
# unit test/development
df.head()

,rec_date,pharm_code,pharm_npi,transtype,pharm_transaction_id,trans_seq,ref_source,ref_date,program_id,pharmacy_id,pat_last_name,pat_first_name,pat_dob,pat_gender,pat_addr1,pat_addr2,pat_city,pat_state,pat_zip,dx1_code,dx2_code,status_date,status_code,sub_status,pres_last_name,...,shiptracking_num,ship_location,ship_address,ship_city,ship_state,ship_zip,has_medical,primary_coverage_type,primary_payer_name,primary_payer_type,secondary_coverage_type,secondary_payer_name,secondary_payer_type,plan_paid_amt,pat_copay,copay_assist_amount,oth_payer_amt,xfer_pharmname,msa_patient_id,msa_patient_bmap,__metadata_run_timestamp,__metadata_app_version,__metadata_output_contract,__metadata_transform_timestamp,__metadata_run_id
0,20181024115959,ACCREDO,1346208949,COM,279133432018102401,0,DIRECT,20181019120000,None,27913343,None,None,None,None,None,None,None,None,00,L40.0,None,20181024115959,CANCELLED,OTHER,None,...,None,None,None,None,None,None,Y,MEDICAL,GENERAL DIRECT,COMMERCIAL,None,None,None,None,None,None,None,None,None,NNNNV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
1,20181025115959,ACCREDO,1346208949,COM,278370982018102502,0,DIRECT,20181022120000,None,27837098,None,None,None,F,None,None,None,None,00,L40.0,None,20181025115959,CANCELLED,INSURANCE OON,GREENBERG,...,None,None,None,None,None,None,Y,MEDICAL,BROWN & TOLAND MEDICAL GRP,COMMERCIAL,None,None,None,None,None,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
2,20181029115959,ACCREDO,1346208949,COM,279181482018102903,0,DIRECT,20181024120000,None,27918148,None,None,None,M,None,None,None,None,00,L40.0,None,20181029115959,CANCELLED,OTHER,SCIURBA,...,None,None,None,None,None,None,Y,MEDICAL,GENERAL HORIZON BCBS NJ,COMMERCIAL,None,None,None,None,None,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
3,20181102115959,ACCREDO,1346208949,COM,267244982018110204,0,DIRECT,20181030120000,None,26724498,None,None,None,F,None,None,None,None,00,Q84,L40.0,20181102115959,CANCELLED,INSURANCE OON,KNUCKLES,...,None,None,None,None,None,None,Y,MEDICAL,ANTHEM BCBS OF KENTUCKY,MEDICARE,None,None,None,None,None,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
4,20181106115959,ACCREDO,1346208949,COM,160618142018110605,0,DIRECT,20181102120000,None,16061814,None,None,None,F,None,None,None,None,00,696.1,None,20181106115959,CANCELLED,OTHER,KORY,...,None,None,None,None,None,None,Y,MEDICAL,EXPRESS SCRIPTS,COMMERCIAL,None,None,None,None,None,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3


# <font color=red>**CALL**</font> THE TRANSFORM

In [544]:
### Use the variables above to execute your transformation.
### the final output needs to be a variable named final_dataframe
logger.info("CALL THE TRANSFORM - execute your transformation - the final output needs to be a variable named final_dataframe")

final_dataframe, final_fail, go = transform.master_patient_status(df)

if go==True:
    logger.info("CALL THE TRANSFORM -  go no go = GO")
elif go==False:
    logger.info("CALL THE TRANSFORM -  go no go = NO go")
else:
    go=False
    logger.info("CALL THE TRANSFORM -  go no go = unknown make it NO go")
    
    

2019-07-26 19:44:20,688 - core.transforms.master.master_patient_status - INFO - CALL THE TRANSFORM - execute your transformation - the final output needs to be a variable named final_dataframe
2019-07-26 19:44:20,695 - core.transforms.master.master_patient_status - INFO - try:
2019-07-26 19:44:20,696 - core.transforms.master.master_patient_status - INFO - Gold Domain List:['ACTIVE', 'CANCELLED', 'DISCONTINUED', 'DENIED', 'PENDING']
2019-07-26 19:44:20,732 - core.transforms.master.master_patient_status - INFO - df in          rec_date pharm_code   pharm_npi transtype pharm_transaction_id  \
0  20181024115959    ACCREDO  1346208949       COM   279133432018102401   
1  20181025115959    ACCREDO  1346208949       COM   278370982018102502   
2  20181029115959    ACCREDO  1346208949       COM   279181482018102903   
3  20181102115959    ACCREDO  1346208949       COM   267244982018110204   
4  20181106115959    ACCREDO  1346208949       COM   160618142018110605   

  trans_seq ref_source     

### *<font color=grey>unittest python*</font>

In [545]:
import unittest

def ut_shape(final_dataframe,df):
    """
    assertion will change based on coding state
    """
    return final_dataframe.shape == df.shape

class TestNotebook(unittest.TestCase):
    
    def test_ut_shape(self):
        
        self.assertEqual(ut_shape(final_dataframe,df),True)
                
"""
expect ok     9:45 am 2019 23 july since transform function
is doing nothing yet
expect FAIL   UNIT TEST 1 as input
expect ok for UNIT TEST 4 file, no fails
expect ok for UNIT TEST 5 file, no fails
expect FAIL for run_id = 3 (all parquet files from the individual unit testing)
"""
# for development only
unittest.main(argv=[''], verbosity= 2, exit=False)        
    

test_ut_shape (__main__.TestNotebook) ... FAIL

FAIL: test_ut_shape (__main__.TestNotebook)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-545-032e27383c72>", line 13, in test_ut_shape
    self.assertEqual(ut_shape(final_dataframe,df),True)
AssertionError: False != True

----------------------------------------------------------------------
Ran 1 test in 0.005s

FAILED (failures=1)


# **publish**
### Writing to S3
Invoke the `publish()` command to write to a given contract. Some things to know:
- To invoke publish a contract must be at the grain of dataset. This is because file names will be set by the dataframe=\>parquet conversion. 
- publish only accepts a pandas dataframe.
- publish does not allow for timedelta data types at this time (this is missing functionality in pyarrow).
- publish handles partitioning the data as per contract, creating file paths, and creating the binary parquet files in S3, as well as the needed metadata. <br>
**- by default, all datasets include a single partition, \_\_metadata\_run\_id, the RunEvent ID of an executed pipeline**

In [546]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
if go==True:
    logger.info("PUBLISH - that's it - its a GO - just provide the final dataframe to the var final_dataframe and we take it from there")
    transform.publish_contract.publish(final_dataframe, run_id, session)
elif go==False:
    logger.info("PUBLISH -  go no go = NO go -  so DONT publish")
else:
    go=False
    logger.info("PUBLISH -  go no go = unknown make it NO go - so DONT publish")    
session.close()

2019-07-26 19:45:22,280 - core.transforms.master.master_patient_status - INFO - PUBLISH - that's it - its a GO - just provide the final dataframe to the var final_dataframe and we take it from there
2019-07-26 19:45:22,282 - core.dataset_contract.DatasetContract - INFO - Publishing dataframe to s3 location s3://ichain-dev/dc-578_patientstatus/sun/ilumya/master/master_patient_status with run ID 3.
2019-07-26 19:45:22,292 - s3parq.publish_parq - INFO - Checking params...
2019-07-26 19:45:22,294 - s3parq.publish_parq - INFO - Params valid.
2019-07-26 19:45:22,316 - s3parq.publish_parq - INFO - Writing to S3...


***